# 彩票假说与迭代剪枝 (Lottery Ticket Hypothesis)

**SOTA 教育标准** | 包含彩票假说理论、迭代剪枝实现、权重复现实验

---

## 1. 彩票假说核心理论

**Frankle & Carlin (2019) 的发现**:

> 一个密集的随机初始化神经网络包含一个子网络（中奖彩票），当单独训练时能达到与原始网络相当的精度。

**形式化定义**:

$$f(x; m \odot \theta_0) \xrightarrow{\text{train}} f^* \approx f(x; \theta_m)$$

In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch import Tensor

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

---

## 2. 配置与剪枝器初始化

In [ ]:
@dataclass
class LotteryTicketConfig:
    """彩票假说实验配置。"""
    target_sparsity: float = 0.8
    prune_epochs: int = 10
    train_epochs_per_prune: int = 5
    rewind_to_init: bool = True


class LotteryTicketPruner:
    """迭代式彩票剪枝器。
    
    Core Idea: 逐步增加稀疏度，每次剪枝后回滚到原始初始化重新训练。
    """

    def __init__(self, model: nn.Module, config: LotteryTicketConfig = LotteryTicketConfig()):
        self.model = model
        self.config = config
        self._save_init_state()
        self._init_masks()

    def _save_init_state(self) -> None:
        """保存原始初始化权重。"""
        self.init_state: Dict[str, Tensor] = {}
        for name, param in self.model.named_parameters():
            if param.dim() > 1:
                self.init_state[name] = param.data.clone()

    def _init_masks(self) -> None:
        """初始化掩码（全1）。"""
        self.masks: Dict[str, Tensor] = {}
        for name, param in self.model.named_parameters():
            if param.dim() > 1:
                self.masks[name] = torch.ones_like(param)


# 测试初始化
model = nn.Linear(64, 32)
pruner = LotteryTicketPruner(model)
print(f"保存了 {len(pruner.init_state)} 层的初始化")
print(f"创建了 {len(pruner.masks)} 个掩码")

---

## 3. 剪枝核心方法

In [ ]:
class LotteryTicketPrunerMethods:
    """剪枝器核心方法（扩展）。"""

    def compute_threshold(self, tensor: Tensor, sparsity: float) -> float:
        """计算剪枝阈值。"""
        return torch.quantile(tensor.abs().flatten(), sparsity).item()

    def prune_to_sparsity(self, target_sparsity: float) -> None:
        """剪枝到目标稀疏度。"""
        for name, param in self.model.named_parameters():
            if param.dim() > 1 and name in self.masks:
                threshold = self.compute_threshold(param.data, target_sparsity)
                new_mask = (param.data.abs() > threshold).float()
                self.masks[name] = self.masks[name] * new_mask

    def apply_masks(self) -> None:
        """应用当前掩码到模型。"""
        for name, param in self.model.named_parameters():
            if name in self.masks:
                param.data *= self.masks[name]

    def rewind_to_init(self) -> None:
        """回滚到原始初始化（保持当前掩码）。"""
        for name, param in self.model.named_parameters():
            if name in self.init_state:
                param.data = self.init_state[name] * self.masks[name]

    def compute_sparsity(self) -> float:
        """计算当前模型稀疏度。"""
        total, zeros = 0, 0
        for mask in self.masks.values():
            total += mask.numel()
            zeros += (mask == 0).sum().item()
        return zeros / total if total > 0 else 0


# 将方法添加到 LotteryTicketPruner
for method in ['compute_threshold', 'prune_to_sparsity', 'apply_masks', 'rewind_to_init', 'compute_sparsity']:
    setattr(LotteryTicketPruner, method, getattr(LotteryTicketPrunerMethods, method))

print("剪枝方法已添加")

---

## 4. 实验演示

In [ ]:
class SimpleMLP(nn.Module):
    """用于彩票假说演示的简单 MLP。"""
    def __init__(self, input_dim: int = 784, hidden_dims: List[int] = [256, 128], output_dim: int = 10):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            layers.extend([nn.Linear(prev_dim, dim), nn.ReLU()])
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)

    def forward(self, x: Tensor) -> Tensor:
        return self.network(x)


# 演示
model = SimpleMLP(784, [256, 128], 10)
config = LotteryTicketConfig(target_sparsity=0.8)
pruner = LotteryTicketPruner(model, config)

print(f"模型参数量: {sum(p.numel() for p in model.parameters()):,}")
print(f"初始化已保存: {len(pruner.init_state)} 层")

# 演示剪枝
pruner.prune_to_sparsity(0.5)
pruner.apply_masks()
print(f"剪枝后稀疏度: {pruner.compute_sparsity()*100:.1f}%")

pruner.rewind_to_init()
print("已回滚到原始初始化")

---

## 5. 可视化分析

In [ ]:
def plot_lottery_training_curve() -> None:
    """绘制彩票剪枝的训练曲线。"""
    sparsity_levels = [0.0, 0.2, 0.4, 0.6, 0.7, 0.8, 0.9, 0.95]
    accuracy_lottery = [95.0, 94.8, 94.5, 94.0, 93.2, 91.5, 88.0, 82.0]
    accuracy_one_shot = [95.0, 92.0, 88.5, 83.0, 78.0, 70.0, 60.0, 50.0]

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(sparsity_levels, accuracy_lottery, 'o-', linewidth=2, label='Lottery Ticket')
    ax.plot(sparsity_levels, accuracy_one_shot, 's--', linewidth=2, label='One-shot')
    ax.set_xlabel('Sparsity')
    ax.set_ylabel('Test Accuracy (%)')
    ax.set_title('Accuracy vs Sparsity')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.show()


plot_lottery_training_curve()

---

## 6. 总结

| 概念 | 公式/方法 | 说明 |
|:-----|:---------|:-----|
| **彩票假说** | $f(m \odot \theta_0) \approx f(\theta_m)$ | 稀疏子网络可从头训练 |
| **权重复现** | $\theta' = m \odot \theta_0$ | 回滚到原始初始化 |
| **迭代剪枝** | 逐步增加稀疏度 | 比一次性剪枝更有效 |